<a href="https://colab.research.google.com/github/Quasimo6688/Open_Hub/blob/main/AI_Build_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**服务器端部署7B模型**

1.装载环境

In [2]:
pip install transformers

In [3]:
pip install PyPDF2

1.装载谷歌云端存储库

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


3.导入模型到服务器并构建基本问答逻辑

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# 设定设备为CPU
device = torch.device("cpu")

# 加载模型和分词器
model_info = "BAAI/AquilaChat-7B"
tokenizer = AutoTokenizer.from_pretrained(model_info, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_info, trust_remote_code=True)

# 动态量化模型
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

# 将模型设置为评估模式并移动到指定设备
quantized_model.eval()
quantized_model.to(device)

# 输入文本
text = "请给出10个要到北京旅游的理由。"
tokens = tokenizer.encode_plus(text)['input_ids'][:-1]
tokens = torch.tensor(tokens)[None,].to(device)

# 定义停止词
stop_tokens = ["###", "[UNK]", "</s>"]

# 禁用梯度计算并生成输出
with torch.no_grad():
    out = quantized_model.generate(tokens, do_sample=True, max_length=512, eos_token_id=100007, bad_words_ids=[[tokenizer.encode(token)[0] for token in stop_tokens]])[0]
    out = tokenizer.decode(out.cpu().numpy().tolist())
    print(out)
